<a href="https://colab.research.google.com/github/khalil192/snake-game-bot/blob/master/snake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import csv
import math
def dist1(curr , next):
      return math.pow(abs(curr[0] - next[0]),2) + math.pow(abs(curr[1] - next[1]),2)

def is_near1(curr , next , head) :
    dist1 = dist(curr , head)
    dist2 = dist(next, head)
    return True if dist2 < dist1 else False

def generateAnInstance1(grid_width , grid_height):
    head_x = np.random.randint(grid_width)
    head_y = np.random.randint(grid_height)
    grid_generated = [ [0 for x in range(grid_width) ] for y in range(grid_height)]
    len_req = np.random.randint((grid_height+ grid_width))
    len_generated = 0
    chk = 0
    tail_x = head_x
    tail_y = head_y
    grid_generated[head_x][head_y] = 2
    len_actually_generated = 1
    while len_generated < len_req :
        moves = []
        if (tail_x-1 >=0 and grid_generated[tail_x-1][tail_y]==0):
            moves.append('l')
        if (tail_x+1 < grid_width and grid_generated[tail_x+1][tail_y]==0):
            moves.append('r')
        if (tail_y-1 >=0 and grid_generated[tail_x][tail_y-1]==0):
            moves.append('u')
        if (tail_y+1 < grid_height and grid_generated[tail_x][tail_y+1]==0):
            moves.append('d')
        if(len(moves)==0) : break
        move = np.random.choice(moves)
        if(move == 'l') : tail_x-=1
        if(move == 'r') : tail_x+=1
        if(move == 'u') : tail_y-=1
        if(move == 'd') : tail_y+=1
        grid_generated[tail_x][tail_y] =1
        len_generated +=1
        len_actually_generated+=1
    #now check if any move leads to death .. 
    label_generated = [0,0,0,0] #l , r,  u, d
    # print('head= ', head_x ,' ',head_y)
    midx = grid_width/2
    midy = grid_height/2
    if(head_x -1 >=0 and grid_generated[head_x-1][head_y] == 0):
        label_generated[2] = 3000
        if(is_near([head_x , head_y] , [head_x-1 , head_y] , [midx , midy])):
            label_generated[2] += 10000
        else: label_generated[2] -=2000
    if(head_x + 1 < grid_width and grid_generated[head_x+1][head_y] == 0):
        label_generated[3] = 3000
        if(is_near([head_x , head_y] , [head_x+1 , head_y] , [midx , midy])):
            label_generated[3] +=10000
        else :label_generated[3] -=2000

    if(head_y -1 >=0 and grid_generated[head_x][head_y-1] == 0):
        label_generated[0] = 3000
        if(is_near([head_x , head_y] , [head_x , head_y-1] , [midx , midy])):
            label_generated[0] += 10000
        else: label_generated[0] -=2000
      
    if(head_y + 1 < grid_height and grid_generated[head_x][head_y+1] == 0):
        label_generated[1] = 3000
        if(is_near([head_x , head_y] , [head_x, head_y+1] , [midx , midy])):
            label_generated[1] += 10000
        else :label_generated[1] -=2000
    #surrondValues
    surrond = [0 for x in range(8)]
    #get surronding values now
    indx = 0
    for x in range(-1,2):
        for y in range(-1,2):
            if(x ==0 and y ==0): continue
            if(head_x+x <0 or head_x+x >= grid_width or head_y+y < 0 or head_y+y >= grid_height or grid_generated[head_x+x][head_y+y]==1):
                surrond[indx] =1     
            indx+=1
    
    sum = 0
    for i in label_generated:
        sum += i
    sum = max(sum , 1)
    for i in range(4):
        label_generated[i]/=sum
    x_diff = head_x - tail_x
    y_diff = head_y - tail_y
    # for row in grid_generated :
    #     print(row)
    # print('label probabilty is :' , label_generated )
    directions = [1 for x in range(4)] #left , right , up,down ,are blocked or not
    if head_y-1 >=0 and grid_generated[head_x][head_y-1] == 0:
            directions[0] = 0
    if head_y+1 < grid_height and grid_generated[head_x][head_y+1] == 0:
            directions[1] = 0
    if head_x-1 >=0 and grid_generated[head_x-1][head_y] == 0:
            directions[2] = 0
    if head_x+1 < grid_width and grid_generated[head_x+1][head_y] == 0:
            directions[3] = 0
    returningList = surrond
    returningList += [head_x / (grid_width) , head_y/(grid_height)]
    returningList += [ len_actually_generated/(grid_width + grid_height)]
    # ,x_diff/grid_width , y_diff/grid_height]
    returningList += [label_generated[0] , label_generated[1] , label_generated[2],label_generated[3]]
    return returningList
# with open('train.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
    # writer.writerow(["leftB" ,"rightB","upB","downB","length", "head_x" , "head_y","x_diff" , "y_diff","label_l" , "label_r" , "label_u", "label_d"])
    
def prepare_data1(filename , rows):
  for i in range(rows):
      data = generateAnInstance(10,10)
      with open( filename, 'a+', newline='') as file:
          writer = csv.writer(file)
          writer.writerow(data)

In [0]:
import numpy as np
import csv
import math
def dist(curr , next):
      return math.pow(abs(curr[0] - next[0]),2) + math.pow(abs(curr[1] - next[1]),2)

def is_near(curr , next , head) :
    dist1 = dist(curr , head)
    dist2 = dist(next, head)
    return True if dist2 < dist1 else False


def check(ls , cell):
    return cell in ls


def generateAnInstance(grid_width , grid_height):
    head_x = np.random.randint(grid_width)
    head_y = np.random.randint(grid_height)
    # grid_generated = [ [0 for x in range(grid_width) ] for y in range(grid_height)]
    len_req = np.random.randint((grid_height+ grid_width))
    len_generated = 0
    chk = 0
    tail_x = head_x
    tail_y = head_y
    len_actually_generated = 1
    snake_quque = [[head_x , head_y]]
    while len_generated < len_req :
        allowed_moves = []
        if (tail_x-1 >=0 and check(snake_quque,[tail_x-1 , tail_y]) == False):
            allowed_moves.append('u')
        if (tail_x+1 < grid_width and check(snake_quque,[tail_x+1 , tail_y]) == False):
            allowed_moves.append('d')
        if (tail_y-1 >=0 and check(snake_quque,[tail_x , tail_y-1]) == False):
            allowed_moves.append('l')
        if (tail_y+1 < grid_height and check(snake_quque,[tail_x , tail_y+1]) == False):
            allowed_moves.append('r')
        if(len(allowed_moves)==0) : break
        move = np.random.choice(allowed_moves)
        if(move == 'l') : tail_y-=1
        if(move == 'r') : tail_y+=1
        if(move == 'u') : tail_x-=1
        if(move == 'd') : tail_x+=1
        # grid_generated[tail_x][tail_y] =1
        snake_quque.append([tail_x , tail_y])
        len_generated +=1
        len_actually_generated+=1
    #now check if any move leads to death .. 
    label_generated = [0,0,0,0] #l , r,  u, d
    # print('head= ', head_x ,' ',head_y)
    midx = grid_width/2
    midy = grid_height/2
    if(head_x -1 >=0 and check(snake_quque , [head_x-1,head_y])==False): #up
        label_generated[2] = 3000
        if(is_near([head_x , head_y] , [head_x-1 , head_y] , [midx , midy])):
            label_generated[2] += 10000
        else: label_generated[2] -=2000
    if(head_x + 1 < grid_width and check(snake_quque,[head_x+1 , head_y]) == False):#down
        label_generated[3] = 3000
        if(is_near([head_x , head_y] , [head_x+1 , head_y] , [midx , midy])):
            label_generated[3] +=10000
        else :label_generated[3] -=2000

    if(head_y -1 >=0 and check(snake_quque , [head_x , head_y-1]) == False): #Left
        label_generated[0] = 3000
        if(is_near([head_x , head_y] , [head_x , head_y-1] , [midx , midy])):
            label_generated[0] += 10000
        else: label_generated[0] -=2000
      
    if(head_y + 1 < grid_height and check(snake_quque , [head_x ,head_y+1]) == False):#right
        label_generated[1] = 3000
        if(is_near([head_x , head_y] , [head_x, head_y+1] , [midx , midy])):
            label_generated[1] += 10000
        else :label_generated[1] -=2000
    #surrondValues
    surrond = [0 for x in range(8)]
    #get surronding values now
    indx = 0
    for x in range(-1,2):
        for y in range(-1,2):
            if(x ==0 and y ==0): continue
            if(head_x+x <0 or head_x+x >= grid_width or head_y+y < 0 or head_y+y >= grid_height or 
                check(snake_quque , [head_x+x,head_y+y])==True):
                surrond[indx] =1     
            indx+=1
    sum = 0
    for i in label_generated:
        sum += i
    sum = max(sum , 1)
    for i in range(4):
        label_generated[i]/=sum
    x_diff = head_x - tail_x
    y_diff = head_y - tail_y
    # for row in grid_generated :
    #     print(row)
    # print('label probabilty is :' , label_generated )
   
    returningList = surrond
    returningList += [head_x / (grid_width) , head_y/(grid_height)]
    returningList += [ len_actually_generated/(grid_width + grid_height)]
    # ,x_diff/grid_width , y_diff/grid_height]
    returningList += [label_generated[0] , label_generated[1] , label_generated[2],label_generated[3]]
    return returningList
# with open('train.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
    # writer.writerow(["leftB" ,"rightB","upB","downB","length", "head_x" , "head_y","x_diff" , "y_diff","label_l" , "label_r" , "label_u", "label_d"])
    
def prepare_data(filename , rows):
  for i in range(rows):
      data = generateAnInstance(10,10)
      with open( filename, 'a+', newline='') as file:
          writer = csv.writer(file)
          writer.writerow(data)

In [0]:
prepare_data('train.csv' , 10000)

In [35]:
import pandas as pd

df = pd.read_csv('train.csv')
X = df.iloc[:, 0:11]  
Y = df.iloc[:,11:15]
print(X.head())
print(Y.head())

   1  1.1  1.2  0  1.3  0.1  1.4  1.5  0.0  0.8  0.25
0  0    0    0  0    1    0    0    1  0.6  0.2  0.75
1  0    0    0  0    1    0    0    0  0.5  0.2  0.90
2  0    0    0  0    1    0    0    1  0.3  0.5  0.30
3  1    1    1  0    0    0    0    0  0.0  0.2  0.05
4  0    0    1  0    1    0    0    0  0.5  0.6  0.40
        1.0     0.0.1     0.0.2     0.0.3
0  0.066667  0.000000  0.866667  0.066667
1  0.333333  0.000000  0.333333  0.333333
2  0.066667  0.000000  0.066667  0.866667
3  0.037037  0.481481  0.000000  0.481481
4  0.866667  0.000000  0.066667  0.066667


In [37]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(52, input_dim=11, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(X, Y, epochs=30, batch_size=10)




Epoch 1/30
9999/9999 [==============================] - 1s 119us/step - loss: 0.6584 - categorical_accuracy: 0.7070
Epoch 2/30
9999/9999 [==============================] - 1s 111us/step - loss: 0.4961 - categorical_accuracy: 0.7951
Epoch 3/30
9999/9999 [==============================] - 1s 110us/step - loss: 0.4920 - categorical_accuracy: 0.7852
Epoch 4/30
9999/9999 [==============================] - 1s 111us/step - loss: 0.4896 - categorical_accuracy: 0.7834
Epoch 5/30
9999/9999 [==============================] - 1s 112us/step - loss: 0.4876 - categorical_accuracy: 0.7857
Epoch 6/30
9999/9999 [==============================] - 1s 111us/step - loss: 0.4862 - categorical_accuracy: 0.7860
Epoch 7/30
9999/9999 [==============================] - 1s 111us/step - loss: 0.4854 - categorical_accuracy: 0.7899
Epoch 8/30
9999/9999 [==============================] - 1s 112us/step - loss: 0.4840 - categorical_accuracy: 0.7912
Epoch 9/30
9999/9999 [==============================] - 1s 112us/step - 

In [0]:
model.save('first_model.h5')

In [0]:
prepare_data('test.csv' , 1000)

In [40]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
model = load_model('first_model.h5')
model.summary()
import pandas as pd

df = pd.read_csv('test.csv')
X = df.iloc[:, 0:11]  
Y = df.iloc[:,11:15]
print(X.head())
print(Y.head())
score = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))




Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 52)                624       
_________________________________________________________________
dense_20 (Dense)             (None, 12)                636       
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 52        
Total params: 1,312
Trainable params: 1,312
Non-trainable params: 0
_________________________________________________________________
   1  1.1  1.2  0  1.3  0.1  0.2  0.3  0.0  0.5  0.35
0  0    0    0  1    0    1    1    0  0.2  0.1  0.80
1  1    1    1  0    0    0    0    0  0.0  0.1  0.05
2  1    1    1  1    0    1    0    0  0.0  0.5  0.30
3  1    1    1  0    0    0    1    1  0.0  0.6  0.50
4  0    0    0  0    0    0    1    0  0.1  0.8  0.25
   0.07142857142857142     0.0.1     0.0.2  0.928571428571